In [1]:
import sys
import praw
from praw.models import MoreComments
import pandas as pd
import numpy as np
import re
from urlextract import URLExtract 
from datetime import datetime, timedelta
from collections import Counter
from urlextract import URLExtract 
from tldextract import extract
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt

In [2]:
import sys
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta
from urlextract import URLExtract 
from tldextract import extract
from IPython.core.display import display, HTML

In [3]:
datetime.now()+timedelta(hours=2)

datetime.datetime(2020, 4, 20, 16, 55, 4, 161855)

In [4]:
covid_regex = '(coronav|covid|pandemi|epidemi|sars)'
fake_regex = '(fake |false ).{0,15}(new\\w|info\\w)'+'|'+\
             '(dis|mis|mal)(info\\w|leading)'+'|'+\
             '(unreli\\w+|unverifi\\w+|untrustw\\w+|fake\\w+)( source)'
fake_yes ='(th.. is|you are|stop spreading|^).{0,15}'+ fake_regex
fake_no = '\\/s$|\\\s$|\\bjk\\b|(not .{0,15})'+fake_regex

In [5]:
def search_corona(text): return bool(re.search(covid_regex, str(text).lower()))
def search_fake(text): return bool(re.search(fake_regex, str(text).lower()))

def search_yes_no(yes, no, text):
    return bool(re.search(yes, str(text).lower())) and not bool(re.search(no, str(text).lower()))    
def search_yes(yes, text):
    return bool(re.search(yes, str(text).lower()))    
def search_no(no, text):
    return not bool(re.search(no, str(text).lower()))    

def has_url(text):
    return URLExtract().has_urls(text)
def extract_domain(url):
    tsd, td, tsu = extract(url)
    return(td + '.' + tsu)
def highlight_re(text, regex):
    htmld = re.sub(r'('+regex+')', '<mark>' + r'\1' + '</mark>', text.lower())
    display(HTML(htmld))

In [6]:
def import_reddit(from_date = '2020-03-01', subreddits = 'all'):
    latest_time = datetime.now()-timedelta(hours=2)
    ###
    ALL = pd.read_csv('http://ec2-3-8-1-23.eu-west-2.compute.amazonaws.com/vlad/all_submissions.csv')
    SR = pd.read_csv("R_selected_subreddits.csv")
    if subreddits != 'all':
        SR = SR[SR.text.isin(list(subreddits))]
    
    ###
    S = pd.read_csv("http://ec2-3-8-1-23.eu-west-2.compute.amazonaws.com/vlad/R_all_submissions.csv")
    S.columns = S.columns.str.strip()
    
    ###
    C = pd.read_csv("http://ec2-3-8-1-23.eu-west-2.compute.amazonaws.com/vlad/R_all_comments.csv")
    C.columns = C.columns.str.strip()
    C['date_created'] = [datetime.fromtimestamp(int(c))-timedelta(hours = 8) for c in C.date_created]

    ###
    CORONA = \
    C.rename(columns = {'id':'comm_id', 'author_id':'comm_author_id','submission_id':'subm_id',\
                    'body':'comm_body', 'score':'comm_score','date_created':'comm_date'}).\
      set_index(['subm_id', 'comm_id']).\
    join(
        ALL.drop('author_id',1)\
           [(ALL.date_created>=from_date) & \
            (ALL.date_created<str(latest_time))].\
           sort_values(['subr_id','date_created']).\
           set_index('subr_id').\
         join(SR.\
               rename(columns = {'id':'subr_id', 'text':'subreddit'})\
               [['subr_id','subreddit']].\
               set_index('subr_id'),\
              on = 'subr_id', how = 'inner').\
              reset_index().set_index(['subr_id','subm_id']).\
         join(S.rename(columns = {'id':'subm_id','title':'subm_title','score':'subm_score','url':'subm_url'})\
              [['subr_id','subm_id','subm_title','subm_score','subm_url']].\
                set_index(['subr_id','subm_id']),\
              how = 'left', on = ['subr_id','subm_id']).\
         reset_index().\
         assign(subm_id = lambda df: 't3_'+ df.subm_id).\
         set_index('subm_id'),\
    on = 'subm_id', how = 'inner')
    
    ###
    print('IMPORTED!\nSubreddits: %d \nSubmissions: %d\nComments: %d' % 
          (SR.shape[0] ,len(set(CORONA.index.get_level_values(level = 0))), CORONA.shape[0]))
    return CORONA

In [32]:
def print_corona_sc(CORONA_DF, n=0, sort = 'newest', rand = False, randseed=np.random.RandomState(), charlim = False):
    ### Filter out long comments
    if charlim>0:
        CORONA_DF = CORONA_DF[[len(b) < charlim for b in CORONA_DF['comm_body']]]
    ### Set n to the number of rows (if 0)
    if n==0: 
        n = CORONA_DF.shape[0]
    ### Sorting
    if sort == 'newest':
        CORONA_DF.sort_values('date_added',ascending = False,inplace = True)
    elif sort == 'most_flagged':
        submission_counts = CORONA_DF['subreddit'].count(level = 'subm_id').rename('flg_count')
        CORONA_DF.join(submission_counts, on ='subm_id').\
        sort_values(['flg_count','date_added'], ascending = [False, False] )
        submissions = pd.Series(CORONA_DF.index.get_level_values('subm_id')).drop_duplicates().tolist()
    ### Extract random sample
    if rand: 
        submissions = pd.Series(submissions).sample(n, random_state = randseed)
        CORONA_DF = CORONA_DF.loc[submissions]
    else:
        submissions = submissions[:n]
    ### Loop to print flagged submissions
    for i, t in enumerate(submissions):
        if i<=n:
            print('\n\n')
            D = CORONA_DF.loc[t]
            print('r/'+D.subreddit[0])
            highlight_re('<b>'+D.subm_title[0]+'</b>', covid_regex)
            print('URL: %s \nPublished: %s ' % (D.subm_url[0], D.date_created[0]))
            for c in D.iterrows():
                highlight_re('<p style="margin-left: 40px"> -> (score: %d) %s </p><br>' % 
                             (c[1].comm_score ,c[1].comm_body), fake_regex)

## 1. Importing Submissions and Comments

In [9]:
COMM = import_reddit(from_date = '2020-04-01')

IMPORTED!
Subreddits: 14 
Submissions: 30378
Comments: 213903


In [13]:
max(COMM.date_added)

'2020-04-19 13:33:56.664875'

In [14]:
def filter_comm(DF, subm_regex_yes, comm_regex_yes, subm_regex_no = 'noneXXXnone', comm_regex_no = 'noneXXXnone'):
    FILT = DF[np.logical_and(
                np.logical_and(
                  np.logical_and([search_yes(subm_regex_yes, s) for s in DF.subm_title],\
                                 [search_no(subm_regex_no, s) for s in DF.subm_title]),\
                  [search_yes(comm_regex_yes, c) for c in DF.comm_body ]),\
              [search_no(comm_regex_no, c) for c in DF.comm_body ])]
    return FILT

In [15]:
covid_regex = '(coronav|covid|pandemi|epidemi|sars)'
fake_regex = '(fake |false).{0,15}(new\\w|info\\w|hood\\w)'+'|'+\
             '(dis|mis|mal)(info\\w|leading)'+'|'+\
             'bs|bullshit'+\
             '(not ).{0,5}(true\\b|verif\\w|real\\b)'+'|'+\
             '(unreli\\w+|unverifi\\w+|untrustw\\w+|fake\\w+)( source)'
fake_regex_no = '((no|not) .{0,10}('+fake_regex+'))'+'|(users often report submissions from this site)'
fake_regex_no

'((no|not) .{0,10}((fake |false).{0,15}(new\\w|info\\w|hood\\w)|(dis|mis|mal)(info\\w|leading)|(not ).{0,5}(true\\b|verif\\w|real\\b)|(unreli\\w+|unverifi\\w+|untrustw\\w+|fake\\w+)( source)))|(users often report submissions from this site)'

In [ ]:
CORONAFLAG = filter_comm(COMM,\
                         subm_regex_yes = covid_regex,\
                         comm_regex_yes = fake_regex,\
                         subm_regex_no = fake_regex,\
                         comm_regex_no = fake_regex_no)
CORONAFLAG.head()

Assign variable to one that has flag

In [17]:
### This does not work yet
COMM['has_flag'] = False
COMM.loc[CORONAFLAG.index,'has_flag'] = True
Counter(COMM['has_flag'])

Counter({False: 213021, True: 882})

In [ ]:
Counter(COMM.subreddit[COMM.has_flag])

### Getting flags using regex

In [21]:
%%capture --no-stdout  --no-display
print_corona_sc(CORONAFLAG, 100, charlim = 500)




r/Coronavirus


URL: https://bdkhealth.com/1-in-1000-new-york-city-residents-have-now-died-from-coronavirus/ 
Published: 2020-04-19 13:08:25.000000 





r/politics


URL: https://www.cnbc.com/2020/04/18/coronavirus-scenes-of-protests-across-the-country-demanding-states-reopen-the-economy.html 
Published: 2020-04-19 12:37:39.000000 





r/politics


...

URL: https://www.medrxiv.org/content/10.1101/2020.04.11.20061473v1 
Published: 2020-04-17 14:17:55.000000 





r/Coronavirus


URL: https://wattsupwiththat.com/2020/04/15/how-to-re-open-the-country-and-control-covid-19-coronavirus/ 
Published: 2020-04-17 14:06:57.000000 





r/politics


URL: https://foreignpolicy.com/2020/04/17/will-trumps-decision-to-cut-who-funding-accomplish-anything/ 
Published: 2020-04-17 13:37:50.000000 


## Find keywords for submissions

In [369]:
S = pd.read_csv("http://ec2-3-8-1-23.eu-west-2.compute.amazonaws.com/vlad/R_all_submissions.csv")
S = pd.merge(S, ALL03[['subm_id','date_created','date_added']].rename(columns = {'subm_id':'id'}), on = 'id')
S['id']='t3_'+ S['id']
S['domain']= [extract_domain(url) for url in S['url']]
S.columns = S.columns.str.lstrip()

In [39]:
import spacy
nlp = spacy.load('en_core_web_sm')
docs = [nlp(text) for text in S.title]

NameError: name 'S' is not defined

In [440]:
chunks = [chunk.text.lower() for doc in docs for chunk in doc.noun_chunks if not chunk.root.is_stop and not chunk.root.is_punct]
common_chunks = Counter(chunks).most_common(300)
with open('submissions_covid_topwords.txt', 'w') as the_file:
    for word, n in common_chunks:
        the_file.write(word + ', ' + str(n)+'\n')

In [ ]:
ventilators, lockdown, crisis, quarantine, virus, masks, outbreak, hospitals, lockdown, deaths, spread, social distancing, izolation, medical supplies, patients, treatment, vaccine, cdc

## Find keywords for comments

In [522]:
def extract_pos(text, df=False):
    pos_list = [(t.text, t.lemma_, t.pos_, t.tag_, t.dep_, list(t.ancestors),list(t.children)) for t in nlp(text)]
    if df:
        return(pd.DataFrame(pos_list, columns=['text','lemma','pos','tag','dep','ancestors','children']))   
    else:
        return(pos_list)

In [398]:
def children_in(tok, wordlist):
    value = False
    for a in tok.children:
        if a.lower_ in wordlist:
            value = a.lower_
            break
    return(value)

In [594]:
def fake_sent_pos(sent):
    v,s,o,neg = [False]*4
    final_phrase = False
    for tok in sent:
        if tok.dep_ == "ROOT":
            if tok.lemma_ in predicates:            
                v = tok.text
                for c in tok.children:
                    if (c.dep_ == "neg"):
                        neg = True
                    elif (c.dep_ == "nsubj") and (c.lower_ in subjects):
                        s = c.text
                    elif c.dep_ in ["dobj","attr","acomp"]:
                        if (c.lower_ in fullobjects):                            
                            o = c.text
                        elif (c.lower_ in partialobjects) and children_in(c, partialattributes):
                            o = children_in(c, partialattributes) + ' ' + c.text
                    elif c.lower_ == 'like' and c.dep_ == "prep":
                        for like in c.children:
                            if (like.lower_ in fullobjects):                            
                                o = 'like ' + like.text
                            elif (like.lower_ in partialobjects) and children_in(like, partialattributes):
                                o = 'like ' + children_in(like, partialattributes) + ' ' + like.text
                            if o and not s:
                                s=''
                if v and o and not neg:
                    if s:
                        final_phrase = '%s %s %s' % (s, v, o)
                    elif s=='':
                        final_phrase = '%s %s' % (v, o)
            elif tok.pos_ in ['NOUN','PROPN']:
                if tok.lower_ in fullobjects:
                    final_phrase = tok.text
                elif tok.lower_ in partialobjects and children_in(tok, partialattributes):
                    final_phrase = children_in(tok, partialattributes) + ' ' + tok.lower_
    return(final_phrase) 

In [589]:
fake_sent_pos(list(nlp(text).sents)[0])

'feels like fake news'

In [400]:
CORONASUB = filter_comm(COMM,\
                         subm_regex_yes = covid_regex,\
                         comm_regex_yes = '.*',\
                         subm_regex_no = fake_regex,\
                         comm_regex_no = '###notatext###')
CORONASUB.shape

(96600, 12)

In [442]:
CORONASUB.loc[CORONASUB.index,'flag_pos']=None
CORONASUB = CORONASUB.sort_values('date_created', ascending = False)
CORONASUB.head()

comm_author_id  \
subm_id   comm_id                     
t3_g47qcb t1_fnvsi6g      t2_5it695   
          t1_fnvs527       t2_armmh   
          t1_fnvszh7    t2_44c7519r   
          t1_fnvvw2i    t2_30qgvy61   
          t1_fnvs12p       t2_6l4z3   

                                                              comm_body  \
subm_id   comm_id                                                         
t3_g47qcb t1_fnvsi6g  Lmao, do they all really dress like school gir...   
          t1_fnvs527                                               Duh?   
          t1_fnvszh7  Many of these women have been groomed and abus...   
          t1_fnvvw2i           I thought they only had sex with robots.   
          t1_fnvs12p  Automatic translation [here](https://translate...   

                      comm_score           comm_date   subr_id  \
subm_id   comm_id                                                
t3_g47qcb t1_fnvsi6g         -13 2020-04-19 07:39:24  t5_2x4yx   
          t1_fnvs527           1 2020-04-19 07:34:59  t5_2x4yx   
          t1_fnvszh7          19 2020-04-19 07:45:10  t5_2x4yx   
          t1_fnvvw2i           4 2020-04-19 08:20:05  t5_2x4yx   
          t1_fnvs12p           2 2020-04-19 07:33:36  t5_2x4yx   

                                    date_created                  date_added  \
subm_id   comm_id                                                              
t3_g47qcb t1_fnvsi6g  2020-04-19 13:33:36.000000  2020-04-19 13:33:53.900038   
          t1_fnvs527  2020-04-19 13:33:36.000000  2020-04-19 13:33:53.900038   
          t1_fnvszh7  2020-04-19 13:33:36.000000  2020-04-19 13:33:53.900038   
          t1_fnvvw2i  2020-04-19 13:33:36.000000  2020-04-19 13:33:53.900038   
          t1_fnvs12p  2020-04-19 13:33:36.000000  2020-04-19 13:33:53.900038   

                        subreddit  \
subm_id   comm_id                   
t3_g47qcb t1_fnvsi6g  Coronavirus   
          t1_fnvs527  Coronavirus   
          t1_fnvszh7  Coronavirus   
          t1_fnvvw2i  Coronavirus   
          t1_fnvs12p  Coronavirus   

                                                             subm_title  \
subm_id   comm_id                                                         
t3_g47qcb t1_fnvsi6g  Sex work in Japan crippled by effects of coron...   
          t1_fnvs527  Sex work in Japan crippled by effects of coron...   
          t1_fnvszh7  Sex work in Japan crippled by effects of coron...   
          t1_fnvvw2i  Sex work in Japan crippled by effects of coron...   
          t1_fnvs12p  Sex work in Japan crippled by effects of coron...   

                      subm_score  \
subm_id   comm_id                  
t3_g47qcb t1_fnvsi6g           8   
          t1_fnvs527           8   
          t1_fnvszh7           8   
          t1_fnvvw2i           8   
          t1_fnvs12p           8   

                                                               subm_url  \
subm_id   comm_id                                                         
t3_g47qcb t1_fnvsi6g  https://www.cnn.com/videos/world/2020/04/19/se...   
          t1_fnvs527  https://www.cnn.com/videos/world/2020/04/19/se...   
          t1_fnvszh7  https://www.cnn.com/videos/world/2020/04/19/se...   
          t1_fnvvw2i  https://www.cnn.com/videos/world/2020/04/19/se...   
          t1_fnvs12p  https://www.cnn.com/videos/world/2020/04/19/se...   

                      has_flag flag_pos  
subm_id   comm_id                        
t3_g47qcb t1_fnvsi6g     False     None  
          t1_fnvs527     False     None  
          t1_fnvszh7     False     None  
          t1_fnvvw2i     False     None  
          t1_fnvs12p     False     None

In [593]:
subjects = ['you','it','this','article','title','headline']
predicates = ['be','spread','stop','call',\
              'look','sound','feel','seem','smell']
fullobjects = ['disinformation','misinformation','misleading','bullshit','bs','falsehood','fabrication']
partialobjects = ['news','information','title','source','article']
partialattributes = ['fake','false','misleading','untrustworthy','bogus','bullshit']
###
fake_regex_wrds = 'fake|false\\w*|bogus|'+\
                  'mislead\\w*|disinf\\w*|misinf\\w*|malinf\\w*|'+\
                  'bullshit|\\bbs\\b|'+\
                  'news|info\\w*|title|article|source|submission'

In [595]:
n, iterr = [0,0]
subm_index_old, comm_index_old = ['','']
for index, row in CORONASUB.iterrows():
    iterr+=1
    for sent in nlp(str(row.comm_body)).sents:
        fake_pos = fake_sent_pos(sent)
        if fake_pos:
            n+=1        
            if index[0]!=subm_index_old:
                print('\n\n\n')
                print("%d / %d" % (n, iterr))
                print(row.subreddit)
                print('--------------------')
                highlight_re('<b>'+row.subm_title+'</b>', covid_regex)
                print('URL: %s \nPublished: %s ' % (row.subm_url, row.date_created[:20]))
                subm_index_old = index[0]
            if index[1]!=comm_index_old:
                highlight_re(row.comm_body, fake_regex_wrds)
                comm_index_old = index[1]
            CORONASUB.loc[index,'flag_pos'] = fake_pos            
            print('Q: ' + fake_pos)
    if iterr % 50 == 1:
        print("#", end = '')
print("%d found / %d searched" % (n, iterr))

####



1 / 161
Coronavirus
--------------------


URL: https://bdkhealth.com/1-in-1000-new-york-city-residents-have-now-died-from-coronavirus/ 
Published: 2020-04-19 13:08:25. 


Q: article is misleading


Q: It 's bullshit
####



3 / 354
politics
--------------------


URL: https://thehill.com/homenews/house/493528-pelosi-really-unfortunate-that-protesters-are-pushing-back-against-coronavirus 
Published: 2020-04-19 12:39:06. 


...

URL: https://www.courier-journal.com/story/news/local/joseph-gerth/2020/03/31/mitch-mcconnell-admits-trump-mishandled-coronavirus-pandemic/5095995002/ 
Published: 2020-04-01 02:59:16. 


Q: Headline is misleading
###########



351 / 95473
politics
--------------------


URL: https://www.businessinsider.com/trump-admits-downplayed-coronavirus-i-knew-it-could-be-horrible-2020-3 
Published: 2020-04-01 02:08:56. 


Q: Bullshit


Q: Bullshit
######################352 found / 96600 searched


In [499]:
Counter(CORONASUB.flag_pos).most_common(10)

[(None, 96279),
 ('fake news', 69),
 ('Bullshit', 43),
 ('bullshit', 40),
 ('misleading title', 21),
 ('title is misleading', 19),
 ('This is bullshit', 16),
 ('BS', 8),
 ('headline is misleading', 7),
 ('This is misleading', 7)]

In [509]:
Counter(CORONASUB.subreddit[~CORONASUB.flag_pos.isna()]).most_common()

[('Coronavirus', 145),
 ('politics', 83),
 ('worldnews', 65),
 ('worldpolitics', 13),
 ('europe', 10),
 ('uspolitics', 2),
 ('PoliticalDiscussion', 2),
 ('news', 1)]

### Percent sources most often flagged 

Filter: source mentioned at least 10 times and flagged at least 2 times

In [638]:
CORONASUB.subm_title[CORONASUB.has_flag_pos]

subm_id    comm_id   
t3_g47edo  t1_fnw2vme    1 in 1000 New York City residents have now die...
           t1_fnw2poy    1 in 1000 New York City residents have now die...
t3_g470di  t1_fnvub7z    Pelosi: 'Really unfortunate' that protesters a...
t3_g45r03  t1_fnvhfyx    Sunlight kills coronavirus 'quickly', says gov...
t3_g41npm  t1_fnuwacs    China reports 16 new coronavirus cases, lowest...
                                               ...                        
t3_fsrxzf  t1_fm34oow    Ostrich antibodies to block COVID-19 virus - O...
t3_fsrenb  t1_fm33614    China Hints That Its Coronavirus Data Doesn’t ...
t3_fsraus  t1_fm386rd    Gerth: Mitch McConnell admits that Trump misha...
t3_fsqj6u  t1_fm31b85    Trump just admitted he downplayed the threat o...
           t1_fm309up    Trump just admitted he downplayed the threat o...
Name: subm_title, Length: 336, dtype: object

In [633]:
CORONASUB['domain'] = CORONASUB.subm_url.apply(extract_domain)
CORONASUB['has_flag_pos'] = ~CORONASUB.flag_pos.isna()
DOMAINS = CORONASUB.groupby('domain').agg(count_all = ("has_flag_pos","count"), count_flag = ("has_flag_pos","sum"))
DOMAINS['percent_flag'] = 100*DOMAINS.count_flag/DOMAINS.count_all

In [636]:
DOMAINS.query('count_all>9 & count_flag>1').sort_values('percent_flag', ascending = False).head(30)

,count_all,count_flag,percent_flag
domain,,,
pepaja.com,11,2.0,18.181818
nbcmiami.com,16,2.0,12.500000
freebeacon.com,36,3.0,8.333333
rsf.org,27,2.0,7.407407
gmanetwork.com,31,2.0,6.451613
oregonlive.com,32,2.0,6.250000
ksbw.com,34,2.0,5.882353
i24news.tv,43,2.0,4.651163
realclearpolitics.com,72,3.0,4.166667
